In [45]:
!pip install mysql.connector
import pandas as pd
import mysql.connector as sql
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from psycopg2 import connect

 ## Database connection


In [46]:
con = connect(
    host = 'localhost',
    database = 'airlines',
    user = 'postgres',
    password = '8286'
    )

In [47]:
url = URL.create(
    "postgresql",
    username="postgres",
    password="8286",
    host="localhost",
    database="airlines"
)

engine = create_engine(url)

 # Loading frames


In [48]:
def load_raw_data(file_name):
    sciezka_do_pliku = f'..\\data\\raw\\{file_name}.csv'
    df_raw = pd.read_csv(sciezka_do_pliku, sep = ',', decimal = '.')
    df_processed = df_raw.rename(columns = str.lower)
    return df_processed
    pass

 # Load individual files into frames

In [49]:
aircraft_df = load_raw_data('aircraft')

In [50]:
airport_list_df = load_raw_data('airport_list')

In [51]:
airport_weather_df = load_raw_data('airport_weather')

In [53]:
flight_df = load_raw_data('flight')
flight_df = flight_df.drop(['unnamed: 0'], axis = 1)

 # Data export to the database

In [54]:
def export_table_to_db(df, table_name):
    print(f"Loading data into {table_name}")
    sql = f'DROP TABLE IF EXISTS {table_name}'
    engine.execute(sql)
    df.to_sql(
        f'{table_name}', 
        con = engine, 
        if_exists = 'replace', 
        index = True, 
        index_label = 'id',
        chunksize = 100)
    con.commit()
    return print(f'Done for {table_name}')
    pass



 ## Data Upload

 ### Uploading `aircraft_df` to the `aircraft` table

In [55]:
export_table_to_db(aircraft_df, 'aircraft')

Loading data into aircraft
Done for aircraft



### Uploading `airport_weather_df` to table `airport_weather

In [56]:
export_table_to_db(airport_weather_df, 'airport_weather')

Loading data into airport_weather
Done for airport_weather


### Uploading `flight_df` to `flight` table
 > Execution of this cell will be time consuming due to the amount of data in the frame.

In [57]:
export_table_to_db(flight_df, 'flight')

Loading data into flight
Done for flight


### Uploading `airport_list_df` to table `airport_list`

In [58]:
export_table_to_db(airport_list_df, 'airport_list')

Loading data into airport_list
Done for airport_list


 # Checking the correctness of the notebook
Run the code below to verify that this part is done correctly

In [59]:
def test_data_export(table_name, expected_count, expected_schema):
    real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[0][0]
    
    real_schema = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 0", engine)
    real_schema = set(real_schema.columns)

    expected_schema = set(expected_schema)

    diff = real_schema.symmetric_difference(expected_schema)

    assert len(diff) == 0, ('Nie zgadzają się kolumny tabel....'
    f'\tOczekiwano: {expected_schema}'
    f'\tOtrzymano: {real_schema}'
    f'\tRóżnica: {diff}')

    assert expected_count == real_count, f'Nie zgadza się liczba wierszy, oczekiwano {expected_count}, otrzymano {real_count}'

 ## Checking the `aircraft` table

In [60]:
aircraft_expected_count = 7383
aircraft_expected_schema = ['id', 'manufacture_year', 'tail_num', 'number_of_seats']

test_data_export('aircraft', aircraft_expected_count, aircraft_expected_schema)

 ## Checking the `airport_weather` table


In [61]:
airport_weather_expected_count = 46226
airport_weather_expected_schema = ['id', 'station', 'name', 'date', 'awnd', 'prcp', 'snow', 'snwd', 'tavg', 'tmax', 'tmin', 'wdf2', 'wdf5', 'wsf2', 'wsf5', 'wt01', 'wt08', 'wt02',
       'wt03', 'wt04', 'wt09', 'wt06', 'wt05', 'pgtm', 'wt10', 'wesd', 'sn32',
       'sx32', 'psun', 'tsun', 'tobs', 'wt07', 'wt11', 'wt18']

test_data_export('airport_weather', airport_weather_expected_count, airport_weather_expected_schema)

 ## Checking the `flight` table


In [62]:
flight_expected_count = 9251880
flight_expected_schema = ['id', 'month', 'day_of_month', 'day_of_week', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id',
       'crs_dep_time', 'dep_time', 'dep_delay_new', 'dep_time_blk',
       'crs_arr_time', 'arr_time', 'arr_delay_new', 'arr_time_blk',
       'cancelled', 'crs_elapsed_time', 'actual_elapsed_time', 'distance',
       'distance_group', 'year', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay']

test_data_export('flight', flight_expected_count, flight_expected_schema)

 ## Checking the `airport_list` table


In [63]:
aircraft_list_expected_count = 97
aircraft_list_expected_schema = ['id', 'origin_airport_id', 'display_airport_name', 'origin_city_name', 'name']

test_data_export('airport_list', aircraft_list_expected_count, aircraft_list_expected_schema)


In [64]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.


 # Summary
In this notebook, we have uploaded the previously downloaded files to the database. As a result, we have created a central place for their storage, which we will use both in data analysis and in the subsequent construction of the reporting system.